In [1]:
import pandas as pd
import os
from datetime import datetime

Читаем файл

In [2]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ti6Su94955DZ4Tky8EbwifpgZf_dTjpBdiVH0Ukhsq94jZdqoHuUytZsFZKfwpXEUCKRFteJRc9P/pub?gid=889004448&single=true&output=csv')

In [3]:
df

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4
...,...,...,...,...,...,...,...
602,2019-04-02,2019-04-01 22:39:16,click,android,121288,CPM,187.4
603,2019-04-02,2019-04-01 22:40:29,click,android,121288,CPM,187.4
604,2019-04-02,2019-04-01 22:40:31,click,web,121288,CPM,187.4
605,2019-04-02,2019-04-01 22:44:14,click,ios,121288,CPM,187.4


Судя по всему в траты не распределены в зависимости от типа оплаты СPC/CPM. Данные о типе оплаты также не верные (отсутствует тип СPC, хотя оплата за клик происходит именно таким способом). Оплата следующая: 187.4 за 1000 просмотров, 187.4 за 1 клик

In [4]:
new_cost_type = []
new_costs = []
for i in range(df.ad_cost_type.shape[0]):
    if df.loc[i, 'event'] == 'view':
        new_cost_type.append('CPM')
        new_costs.append(0.187)
    else:
        new_cost_type.append('CPC')
        new_costs.append(187.4)
df['cost_type'] = pd.Series(new_cost_type)
df['new_ad_cost'] = pd.Series(new_costs)

In [5]:
df['ad_cost'] = df['ad_cost'].apply(lambda x: x / 1000)

In [6]:
df.drop(['ad_cost_type', 'ad_cost'], axis=1, inplace=True)

In [8]:
df

,date,time,event,platform,ad_id,cost_type,new_ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,0.187
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,0.187
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,0.187
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,0.187
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,0.187
...,...,...,...,...,...,...,...
602,2019-04-02,2019-04-01 22:39:16,click,android,121288,CPC,187.400
603,2019-04-02,2019-04-01 22:40:29,click,android,121288,CPC,187.400
604,2019-04-02,2019-04-01 22:40:31,click,web,121288,CPC,187.400
605,2019-04-02,2019-04-01 22:44:14,click,ios,121288,CPC,187.400


In [9]:
df.dtypes

date            object
time            object
event           object
platform        object
ad_id            int64
cost_type       object
new_ad_cost    float64
dtype: object

In [10]:
pivot_df = df.groupby(['date','event'], as_index=False).agg({'time': 'count', 'new_ad_cost': 'sum'}) \
        .pivot(index = 'date', columns = 'event', values = 'time').reset_index()

In [11]:
pivot_df

event,date,click,view
0,2019-04-01,17,491
1,2019-04-02,6,93


In [12]:
pivot_df = pivot_df.assign(ctr = pivot_df.click / pivot_df.view)

In [13]:
pivot_df

event,date,click,view,ctr
0,2019-04-01,17,491,0.034623
1,2019-04-02,6,93,0.064516


In [14]:
df

,date,time,event,platform,ad_id,cost_type,new_ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,0.187
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,0.187
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,0.187
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,0.187
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,0.187
...,...,...,...,...,...,...,...
602,2019-04-02,2019-04-01 22:39:16,click,android,121288,CPC,187.400
603,2019-04-02,2019-04-01 22:40:29,click,android,121288,CPC,187.400
604,2019-04-02,2019-04-01 22:40:31,click,web,121288,CPC,187.400
605,2019-04-02,2019-04-01 22:44:14,click,ios,121288,CPC,187.400


Добавим в сводную таблицу сведения о расходах

In [15]:
cost_view = df.groupby('date', as_index=False).agg({'new_ad_cost': 'sum'})

In [16]:
union_df = pd.merge(pivot_df, cost_view, on = 'date')

In [17]:
union_df

,date,click,view,ctr,new_ad_cost
0,2019-04-01,17,491,0.034623,3277.617
1,2019-04-02,6,93,0.064516,1141.791


Формируем данные для отчета:

In [18]:
# процентная разница в кликах
diff_click_pct = ((union_df.iloc[1].click - union_df.iloc[0].click) / union_df.iloc[0].click  * 100).round(2)

In [19]:
# процентная разница в просмотрах
diff_view_pct = ((union_df.iloc[1]['view'] - union_df.iloc[0]['view']) / union_df.iloc[0]['view'] * 100).round(2)

In [20]:
# процентная разница в ctr
diff_ctr_pct = ((union_df.iloc[1].ctr - union_df.iloc[0].ctr) / union_df.iloc[0].ctr * 100).round(2)

In [21]:
# процентная разница в затратах
diff_costs_pct = ((union_df.iloc[1].new_ad_cost - union_df.iloc[0].new_ad_cost) /union_df.iloc[0].new_ad_cost * 100).round(2)

In [22]:
click = union_df.iloc[1].click

In [23]:
view = union_df.iloc[1]['view']

In [24]:
ctr = (union_df.iloc[1].ctr * 100).round(2)

In [25]:
costs = union_df.iloc[1].new_ad_cost.round(2)

In [26]:
today_date = datetime.today().strftime('%Y-%m-%d')

In [27]:
file_name = f'basic_metrics {today_date}.txt'

In [28]:
message = f'Отчет по объявлению 121288 за 2 апреля\n \
            Траты: {costs} рублей ({diff_costs_pct}%)\n \
            Показы: {view} ({diff_view_pct}%)\n \
            Клики: {click} ({diff_click_pct}%)\n \
            CTR: {ctr} ({diff_ctr_pct}%)'

In [29]:
with open(file_name, 'w', encoding='utf-8') as text_file:
    text_file.write(message)

# Отправка документа в чат Telegram

In [104]:

import requests
import json
from urllib.parse import urlencode


token = '1834296021:AAHH5lX31x7zuKNEICfB6bOEvXbaeUb2oyw'
chat_id = 364563004  # your chat id

message = f'{file_name}'  # text which you want to send

params = {'chat_id': chat_id, 'text': message}

base_url = f'https://api.telegram.org/bot{token}/'

filepath = f'{file_name}'
url = base_url + 'sendDocument?' + urlencode(params)

files = {'document': open(filepath, 'rb')}

resp = requests.get(url, files=files)


Далее используя модуль pyinstaller упаковываем весь скрипт в .exe файл и благополучно используем